In [1]:
from pprint import pprint

<h1 style="color:Tomato;"> Datasets </h1>

The first step always is to get acces to the dataset of interest

When it comes to LLMs, we have thousands of datasets in different formats (CSV, JSON, Arrow, SQL, WebDataset, Parquet) for various tasks 

The ```datasets``` module provides a simple interface to download a specific dataset from thousands of available datasets on HF.

Please visit the official [documentation](https://huggingface.co/docs/datasets/en/index) to know more about the ```datasets``` module

In [ ]:
# if you are using colab, uncomment this code below 
# !pip install datasets

In [6]:
import datasets 

#blank cell to prevent transitioning in slides

<h1 style="color:Tomato;"> Loading a dataset </h1>

Let us first download a small dataset of movie reviews used in supervised learning.

We can visit the [hub](https://huggingface.co/datasets) and search for the dataset.

We found a dataset at this link: https://huggingface.co/datasets/stanfordnlp/imdb.

We can now easily load the dataset by specifying the unique path `stanfordnlp/imdb` as an argument.

When we download the dataset for the first time, it will be cached locally. Since we have already downloaded it, we will load it from the cache directory.

The cached data is stored in a [memory-mapped](https://huggingface.co/docs/datasets/en/about_arrow) columnar format, which uses less memory


Now we can load the dataset using ```load_dataset()``` function.

In [7]:
from datasets import load_dataset
imdb_dataset = load_dataset("stanfordnlp/imdb")
print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


Notice the structure: the "train" and "test" splits are wrapped by the ```DatasetDict``` class

Each split is an instance of the `Dataset` class and contains two fields: _features_ and _num_rows_.

We can think of the ```Dataset``` class as a **table** with 25000 rows and two columns (text,label)

What if we just want the train split?

In [8]:
imdb_train_split = imdb_dataset['train']
print(imdb_train_split)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


It is not a ```DatasetDict``` class (makes sense!)

We also want to remove the third split: _unsupervised_. We can use the pop method to delete the key 'unsupervised'.

In [9]:
_ = imdb_dataset.pop('unsupervised')
print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


blank

What if we want to **download** the **train split** alone from the hub? 

In [10]:
train_split = load_dataset("stanfordnlp/imdb",split="train")
print(train_split)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


We can divide the split further into "**train**" and "**test**" (evaluation) splits (as we do in sklearn)

In [11]:
small_ds = train_split.train_test_split(test_size=0.2) 
print(small_ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})


[Note]: To be skipped in presentation <br>
We can also save the dataset in different formats 

In [ ]:
train_file = small_ds["train"]
test_file = small_ds["test"]

In [ ]:
train_file.to_csv('data/train.csv')

In [ ]:
test_file.to_csv('data/test.csv')

<h1 style="color:Tomato;"> Loading Local Datasets </h1>

Suppose we have our dataset files stored locally in CSV format 

Suppose we have two files, namely "_train.csv_" and "_test.csv_" in a directory named _data_ <br>
<pre>
data <br>
 |--train.csv <br>
 |--test.csv <br>
</pre>

**NOTE:** All the files **should have** the same number of columns (features) with the same column (feature) name.

Then we can simply load them as follows

In [12]:
data_files=["data/train.csv","data/test.csv"]
local_dataset = load_dataset("csv",data_files=data_files)
print(local_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


Click on here for loading files in other supported formats: [here](https://huggingface.co/docs/datasets/en/tabular_load#csv-files)

In [13]:
train_test_splits = local_dataset["train"].train_test_split(test_size=0.2)

We can convert the dataset from any format to **pyarrow**(.arrow) format (that uses less memory) by saving it to the disk

In [14]:
train_test_splits.save_to_disk('pyarrow_dataset/movie_review')

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

It creates a folder named _pyarrow_dataset_ with a sub-folder for each split (namely, train, test if exists) <br>
<pre>
pyarrow_dataset <br>
|-- movie_review <br>
    |--train <br>
        |--data-0000-of-0001.arrow <br>
        |--dataset_info.json <br>
        |--state.json <br>
    |--test <br>
        |--data-00000-of-0001.arrow <br>
        |--dataset_info.json <br>
        |--state.json <br>
</pre>      

Now we can load the dataset using `load_from_disk` function

In [15]:
from datasets import load_from_disk

In [16]:
raw_dataset_from_disk = load_from_disk('pyarrow_dataset/movie_review')
print(raw_dataset_from_disk)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})


skip

<h1 style="color:Tomato;"> Accessing samples </h1>

**Disclaimer:** <br>
This dataset may contain samples with language or content that could be considered inappropriate or offensive. Your discretion is advised when accessing or utilizing this data. The inclusion of such content does not reflect the views or values of the instructor and is intended solely for educaton purposes.

To access samples from the `DatasetDict`, we need to specify the split from which the samples should be retrieved.

Let us access a sample from the _train_ split

In [17]:
idx = 1000
example = imdb_dataset['train'][idx]
pprint(example)

{'label': 0,
 'text': 'Although I have to admit I laughed more watching this movie than the '
         'last few comedies I saw.<br /><br />The budget must have consisted '
         'of pocket change from the actors. The production values are so low '
         'that they actual made it kind of fun to watch. Reminds me of the '
         'Robot Monster made up of a guy in a gorilla suit with a cardboard '
         'diving helmet on.<br /><br />In one scene a hapless victim gets '
         'their arm and leg cut off. Geez, hard to believe but the Black '
         'Knight scene from Holy Grail was more realistic. I kept wondering '
         'why the victim didn\'t start shouting " None Shall Pass" and " It\'s '
         'only a flesh wound, I\'ve had worse". It was one of the funniest '
         'scenes I\'ve seen in the past year.<br /><br />The "gladiator/demon" '
         'was a stitch too. Between the horribly cheap costume and the geeky '
         'look of the guy in it the end result

Note that the returned object is a ```dictionary``` with ```text``` and ```label``` field, not a ```Dataset```. 

In [19]:
example = imdb_dataset['train'].select([idx]) # note, we are passing a list (iterable) of idx 
print(example)

Dataset({
    features: ['text', 'label'],
    num_rows: 1
})


Now it returns an object of the ```Dataset``` class.

We can also select a range of samples by passing a list of indices

In [20]:
idx = range(0,100,2) # take even indexed samples
examples = imdb_dataset['train'].select(idx) # note, we are passing a list (iterable) of idx 
print(examples)

Dataset({
    features: ['text', 'label'],
    num_rows: 50
})


skip

<h1 style="color:Tomato;"> Translation Dataset </h1> 

Let's take a look at the WMT-14 dataset: https://huggingface.co/datasets/wmt/wmt14

Unlike the previous case, now the dataset is composed of 5 **sub-datasets** (called **configurations** in HF datasets): "rs-en" "cs-en","fr-en", "hi-en","de-en"

In addition to the dataset name, we need to mention the subset we wish to download

the ```load_dataset``` function contains a lot of optional arguments for customization [Doc](https://huggingface.co/docs/datasets/v2.19.0/en/package_reference/loading_methods#datasets.load_dataset).

Most commonly used are ```path,name, split, revision, streaming```

We can visit the hub to know the details of the dataset or use the respective functions in the script

In [21]:
from datasets import get_dataset_config_names, get_dataset_split_names
print(get_dataset_config_names("wmt/wmt14"))
print(get_dataset_split_names("wmt/wmt14","hi-en")) # select a subset

['cs-en', 'de-en', 'fr-en', 'hi-en', 'ru-en']
['train', 'validation', 'test']


In [22]:
translation_dataset = load_dataset(path="wmt/wmt14",name="hi-en")
print(translation_dataset)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 32863
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})


skip

If we want all samples as a single dataset (without any splits), we can combine the splits

In [23]:
raw_dataset = load_dataset(path="wmt/wmt14",name="hi-en", split="train+test+validation")
print(raw_dataset)
print(len(raw_dataset))

Dataset({
    features: ['translation'],
    num_rows: 35890
})
35890


Note carefully that no splits are there in the ```Dataset```

<h1 style="color:Tomato;"> Features </h1>

Features defines the internal structure of a dataset. It is used to specify the underlying serialization format. 

The **features** of a dataset vary based on the type of dataset (i.e., task, domain..)

In [24]:
pprint(translation_dataset['train'].features)

{'translation': Translation(languages=['hi', 'en'], id=None)}


In this particular case, the Feature is "Translation"

* Some other feature classes are: Value, ClassLabel, Image, Audio, Array2D, ArrayxD, Sequence
* Visit [here](https://huggingface.co/docs/datasets/en/about_dataset_features) to learn more about handling features
* Let's take a look at one more dataset

<h3 style="color:Tomato;"> Microsoft Research Paraphrase Corpus (MRPC) </h3>

Dataset name: _glue_, configuration name (sub-datasets): _mrpc_

In [26]:
mrpc_dataset = load_dataset('glue', 'mrpc', split='train') # load MRPC from GLUE benchmark
print(mrpc_dataset)

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})


In [27]:
pprint(mrpc_dataset.features)

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}


There are four features

First two features are of `Value` with dtype=strings (texts)

Third feature is a class ```ClassLabel``` for labels that contains label names


The fourth feature is again a `Value` with dtype=integer for representing indices

<h1 style="color:Tomato;"> Common Methods </h1>

Let's look at some helpful methods  of ```DatasetDict``` class such as `.filter`,`.map`,`.concatenate`.

These methods operates on all the samples across the splits

We can make use of multiple cpus for paralleization of task for a larger dataset.

In [28]:
import multiprocessing
print(multiprocessing.cpu_count()) 

12


skip

<h3 style="color:Blue;"> Filtering </h3>

Suppose we want to ensure that the number of words in a sentence should be of at least 100

In [29]:
print('Before Filtering')
print(20*'-')
print(imdb_dataset)

Before Filtering
--------------------
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


skip

In [30]:
num_words = 100
imdb_filtered_dataset = imdb_dataset.filter(lambda example:len(example['text'].split(' '))>=num_words)
print('After Filtering')
print(20*'-')
print(imdb_filtered_dataset)

After Filtering
--------------------
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 22074
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21909
    })
})


skip

<h2 style="color:Blue;"> Map </h2>

One of the most frequently used operations

Task: Let's add the prefix ```IMDB:``` to all the samples

In [31]:
def add_prefix(example):
    example["text"] = "IMDB:"+example["text"]
    return example

skip

In [32]:
imdb_prefixed_dataset = imdb_dataset.map(add_prefix)
print(imdb_prefixed_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


skip

In [33]:
print(imdb_prefixed_dataset["train"]["text"][1000])

IMDB:Although I have to admit I laughed more watching this movie than the last few comedies I saw.<br /><br />The budget must have consisted of pocket change from the actors. The production values are so low that they actual made it kind of fun to watch. Reminds me of the Robot Monster made up of a guy in a gorilla suit with a cardboard diving helmet on.<br /><br />In one scene a hapless victim gets their arm and leg cut off. Geez, hard to believe but the Black Knight scene from Holy Grail was more realistic. I kept wondering why the victim didn't start shouting " None Shall Pass" and " It's only a flesh wound, I've had worse". It was one of the funniest scenes I've seen in the past year.<br /><br />The "gladiator/demon" was a stitch too. Between the horribly cheap costume and the geeky look of the guy in it the end result was hysterical.<br /><br />Truly a movie that is bad enough to be watchable. Kind of like seeing a slow motion auto accident on film.<br /><br />


skip

<h2 style="color:Blue;"> Concatenate datasets </h2>

Often we need to combine two or more datasets (say, Imdb review with rotten tomatoes) to create a larger dataset

This is a typical approach for building pretraining datasets

The only requirement is that the datasets must have **same features and same number of splits**

In [34]:
imdb_dataset_whole = load_dataset("stanfordnlp/imdb",split="train+test") # split="all" will load duplicates in this case
print(imdb_dataset_whole)
print(imdb_dataset_whole.features)

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


skip

In [35]:
rt_dataset_whole = load_dataset("cornell-movie-review-data/rotten_tomatoes",split="all")
print(rt_dataset_whole)
print(rt_dataset_whole.features)

Dataset({
    features: ['text', 'label'],
    num_rows: 10662
})
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


Now, we can combine the datasets by passing those in a list

In [36]:
concat_dataset = datasets.concatenate_datasets([imdb_dataset_whole,rt_dataset_whole],axis=0)
print(concat_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 60662
})


We can then create ```train``` and ```test``` splits by calling ```train_test_split``` (this internally shuffles the samples before splitting)

In [ ]:
# train,test = concat_dataset.train_test_split()

<h2 style="color:Blue;"> Interleaving Datasets </h2>

Often we have $n$ skewed datasets (that is number of samples in the dataset might differ drastically)

So we need to build a new dataset by intervaling the samples from each dataset according to its distribution 

In the above case, IMDB has 50000 samples whereas rotten tomatoes has 10000 samples

Let's build a dataset that contains 60% of samples from IMDB and 40% from rotten tomatoes

In [37]:
from datasets import interleave_datasets
inter_datasets = interleave_datasets([imdb_dataset_whole,rt_dataset_whole],
                                     probabilities=[0.6,0.4])
print(inter_datasets)

Dataset({
    features: ['text', 'label'],
    num_rows: 26876
})


By default, ```stopping_strategy=first_exhausted``` (construction is stopped as soon one of the datasets runs out of sample)

<h1 style="color:Tomato;"> Iterable dataset</h1>


Suitable for loading samples from **large datasets** (hundreads of GB ) iteratively without writing anything to local disk

In [38]:
imdb_iter_dataset = load_dataset("stanfordnlp/imdb",split="train",streaming=True)
print(imdb_iter_dataset)

IterableDataset({
    features: ['text', 'label'],
    n_shards: 1
})


skip

In [39]:
for example in imdb_iter_dataset:
    print(example)
    break

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

The methods like ```map,filter``` works on the fly!

In [40]:
for example in imdb_iter_dataset.map(add_prefix):
    print(example)
    break

{'text': 'IMDB:I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and f

skip

## Questions to Ponder

Can we load a dataset directly from external links?

Can we load a dataset from zipped file directly?